Antes de hacer nada cargamos las librerias que vamos a necesitar

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# Carga y preparación de los datos.

In [2]:
datos = pd.read_csv("churn.csv")

Realizamos un resumen estadistico de los datos

In [3]:
datos.describe()

,Account Length,Area Code,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,101.064806,437.182418,8.099010,179.775098,100.435644,30.562307,200.980348,100.114311,17.083540,200.872037,100.107711,9.039325,10.237294,4.479448,2.764581,1.562856
std,39.822106,42.371290,13.688365,54.467389,20.069084,9.259435,50.713844,19.922625,4.310668,50.573847,19.568609,2.275873,2.791840,2.461214,0.753773,1.315491
min,1.000000,408.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.200000,33.000000,1.040000,0.000000,0.000000,0.000000,0.000000
25%,74.000000,408.000000,0.000000,143.700000,87.000000,24.430000,166.600000,87.000000,14.160000,167.000000,87.000000,7.520000,8.500000,3.000000,2.300000,1.000000
50%,101.000000,415.000000,0.000000,179.400000,101.000000,30.500000,201.400000,100.000000,17.120000,201.200000,100.000000,9.050000,10.300000,4.000000,2.780000,1.000000
75%,127.000000,510.000000,20.000000,216.400000,114.000000,36.790000,235.300000,114.000000,20.000000,235.300000,113.000000,10.590000,12.100000,6.000000,3.270000,2.000000
max,243.000000,510.000000,51.000000,350.800000,165.000000,59.640000,363.700000,170.000000,30.910000,395.000000,175.000000,17.770000,20.000000,20.000000,5.400000,9.000000


Preparación de los datos

In [4]:
datos.columns

Index(['State', 'Account Length', 'Area Code', 'Phone', 'Int'l Plan',
       'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge',
       'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
       'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls', 'Churn'],
      dtype='object')

In [7]:
# Eliminamos las variables que no nos interesan.
# Las columnas 'Phone', 'State' y 'Area Code' ya no están presentes en el DataFrame.

# Transformamos la variable objetivo en 0 y 1 (fuga)
datos['Churn2'] = np.where(datos['Churn'] == "False.", 0, 1)
datos = datos.drop('Churn', axis=1)
datos = datos.rename(columns={'Churn2': 'Churn'})

# Convertimos en numericas aquellas variables categoricas.
# Utlilizamos una forma mas rapida.
datos["Int'l Plan"] = np.where(datos["Int'l Plan"] == 'yes', 1, 0)
datos['VMail Plan'] = np.where(datos['VMail Plan'] == 'yes', 1, 0)

In [7]:
datos.dtypes

,0
Account Length,int64
Int'l Plan,int64
VMail Plan,int64
VMail Message,int64
Day Mins,float64
Day Calls,int64
Day Charge,float64
Eve Mins,float64
Eve Calls,int64
Eve Charge,float64


In [8]:
# Separamos los datos en train y test.
# train para entrenar el modelo.
# test para evaluar los resultados obtenidos.
X = datos.drop('Churn', axis=1)
y = datos['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Realizamos un modelo simple.

En este caso un arbol de decision.

In [9]:
tree = DecisionTreeClassifier(max_depth=2, min_samples_split=10)
tree.fit(X_train, y_train)

# Realizamos la prediccion para los datos de test.
y_pred = tree.predict(X_test)

# Evaluamos los resultados obtenidos.
# Primero realizamos la matriz de confusion.
matrix_arbol = confusion_matrix(y_test, y_pred)
print("Matriz de confusion Arbol:")
print(matrix_arbol)

Matriz de confusion Arbol:
[[834]]


c:\Users\Dell\PyhtonIA\ML Modelos Ensemble\.venv\Lib\site-packages\sklearn\metrics\_classification.py:534: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Analizamos numericamente los resultados.

In [10]:
accuracy_arbol = sum(np.diag(matrix_arbol)) / np.sum(matrix_arbol)
print(f"Precision Arbol: {accuracy_arbol:.4f}")

Precision Arbol: 1.0000


# BAGGING manual

Pasamos a realizar un modelo de bagging casero.

Para entrenar cada uno de los modelos usamos una parte de los datos.
Con cada modelo realizamos una prediccion sobre los datos de test.

Establecemos un sistema de votacion para determinar el resultado final.

Los grupos los podemos crear manualmente o de forma aleatoria.

## Reparto de los datos de train y test manualmente.

In [11]:
X_train.shape

(2499, 17)

In [13]:
# Creamos 5 grupos
datostra1 = X_train.iloc[:500]
datostra2 = X_train.iloc[500:1000]
datostra3 = X_train.iloc[1000:1500]
datostra4 = X_train.iloc[1500:2000]
datostra5 = X_train.iloc[2000:]

y_train1 = y_train.iloc[:500]
y_train2 = y_train.iloc[500:1000]
y_train3 = y_train.iloc[1000:1500]
y_train4 = y_train.iloc[1500:2000]
y_train5 = y_train.iloc[2000:]

Entrenamos los 5 modelos. Cada uno con un grupo de datos.


In [14]:
tree1 = DecisionTreeClassifier(max_depth=2, min_samples_split=10)
tree1.fit(datostra1, y_train1)

tree2 = DecisionTreeClassifier(max_depth=2, min_samples_split=10)
tree2.fit(datostra2, y_train2)

tree3 = DecisionTreeClassifier(max_depth=2, min_samples_split=10)
tree3.fit(datostra3, y_train3)

tree4 = DecisionTreeClassifier(max_depth=2, min_samples_split=10)
tree4.fit(datostra4, y_train4)

tree5 = DecisionTreeClassifier(max_depth=2, min_samples_split=10)
tree5.fit(datostra5, y_train5)

,criterion,'gini'
,splitter,'best'
,max_depth,2
,min_samples_split,10
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


Realizamos las 5 predicciones

In [15]:
prediccion1 = tree1.predict(X_test)
prediccion2 = tree2.predict(X_test)
prediccion3 = tree3.predict(X_test)
prediccion4 = tree4.predict(X_test)
prediccion5 = tree5.predict(X_test)

Juntamos todas las predicciones en una misma tabla.

In [16]:
prediccion_final = np.column_stack((prediccion1, prediccion2, prediccion3, prediccion4, prediccion5))
prediccion_final = pd.DataFrame(prediccion_final, columns=['Pred1', 'Pred2', 'Pred3', 'Pred4', 'Pred5'])
prediccion_final

,Pred1,Pred2,Pred3,Pred4,Pred5
0,1,1,1,1,1
1,1,1,1,1,1
2,1,1,1,1,1
3,1,1,1,1,1
4,1,1,1,1,1
...,...,...,...,...,...
829,1,1,1,1,1
830,1,1,1,1,1
831,1,1,1,1,1
832,1,1,1,1,1


Realizamos una suma de las predicciones.

In [17]:
prediccion_final['Suma'] = prediccion_final['Pred1'] + prediccion_final['Pred2'] + prediccion_final['Pred3'] + prediccion_final['Pred4'] + prediccion_final['Pred5']

In [17]:
prediccion_final

,Pred1,Pred2,Pred3,Pred4,Pred5,Suma
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,1,1,2
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
829,0,0,0,0,0,0
830,0,0,0,0,0,0
831,0,0,0,0,0,0
832,0,0,0,0,0,0


Realizamos la prediccion final como una votacion de las predicciones individuales.

Si 3 o mas modelos determinan que se fuga la respuesta es fuga.

Si menos de tres modelos establecen que se fuga determinamos que NO se fuga.

In [18]:
prediccion_final['voto'] = np.where(prediccion_final['Suma'] > 2, 1, 0)
prediccion_final[5:15]

,Pred1,Pred2,Pred3,Pred4,Pred5,Suma,voto
5,1,1,1,1,1,5,1
6,1,1,1,1,1,5,1
7,1,1,1,1,1,5,1
8,1,1,1,1,1,5,1
9,1,1,1,1,1,5,1
10,1,1,1,1,1,5,1
11,1,1,1,1,1,5,1
12,1,1,1,1,1,5,1
13,1,1,1,1,1,5,1
14,1,1,1,1,1,5,1


In [19]:
# Analizamos los resultados obtenidos.
matrix_bagging1 = confusion_matrix(y_test, prediccion_final['voto'])
print("\nMatriz de confusion Bagging Manual:")
print(matrix_bagging1)

accuracy_bagging1 = sum(np.diag(matrix_bagging1)) / np.sum(matrix_bagging1)
print(f"Precision Bagging Manual: {accuracy_bagging1:.4f}")

# Lo comparamos con el acierto anterior
print(f"\nPrecision Arbol: {accuracy_arbol:.4f}")


Matriz de confusion Bagging Manual:
[[834]]
Precision Bagging Manual: 1.0000

Precision Arbol: 1.0000


c:\Users\Dell\PyhtonIA\ML Modelos Ensemble\.venv\Lib\site-packages\sklearn\metrics\_classification.py:534: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


## Reparto de los datos de train y aleatoria.

In [20]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

modelos = []
predicciones = []

for train_index, test_index in kf.split(X_train):
   print("Separación:", test_index)
   X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
   y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

   modelo = DecisionTreeClassifier(max_depth=2, min_samples_split=10)
   modelo.fit(X_train_fold, y_train_fold)
   modelos.append(modelo)

   prediccion = modelo.predict(X_test)
   predicciones.append(prediccion)

Separación: [  25   29   43   44   48   49   51   56   67   69   70   73   76   84
   93   96  100  111  124  134  135  141  149  157  163  168  170  173
  178  179  188  194  196  199  208  210  211  212  218  220  229  231
  233  237  239  240  246  247  251  252  254  259  279  282  289  291
  296  298  299  318  321  324  332  342  350  353  361  367  368  387
  402  406  407  408  410  414  420  422  430  432  433  435  438  440
  443  445  450  457  461  463  464  471  478  479  480  482  485  486
  495  497  498  507  508  509  518  521  522  527  528  530  532  544
  554  564  565  572  581  582  596  599  605  610  613  617  618  629
  632  637  647  649  650  651  654  662  668  678  679  693  695  700
  707  729  736  742  759  764  765  767  772  781  786  787  788  789
  790  798  800  801  802  808  809  810  812  817  829  836  840  861
  862  879  886  902  903  906  907  911  915  923  930  941  942  944
  949  952  962  978  993 1000 1011 1014 1017 1018 1029 1033 1036

In [21]:
# Juntamos todas las predicciones en una misma tabla.
prediccion_final = np.column_stack(predicciones)
prediccion_final = pd.DataFrame(prediccion_final, columns=['Pred1', 'Pred2', 'Pred3', 'Pred4', 'Pred5'])
prediccion_final

,Pred1,Pred2,Pred3,Pred4,Pred5
0,1,1,1,1,1
1,1,1,1,1,1
2,1,1,1,1,1
3,1,1,1,1,1
4,1,1,1,1,1
...,...,...,...,...,...
829,1,1,1,1,1
830,1,1,1,1,1
831,1,1,1,1,1
832,1,1,1,1,1


In [22]:
# Realizamos una suma de las predicciones.
prediccion_final['Suma'] = prediccion_final['Pred1'] + prediccion_final['Pred2'] + prediccion_final['Pred3'] + prediccion_final['Pred4'] + prediccion_final['Pred5']
prediccion_final

,Pred1,Pred2,Pred3,Pred4,Pred5,Suma
0,1,1,1,1,1,5
1,1,1,1,1,1,5
2,1,1,1,1,1,5
3,1,1,1,1,1,5
4,1,1,1,1,1,5
...,...,...,...,...,...,...
829,1,1,1,1,1,5
830,1,1,1,1,1,5
831,1,1,1,1,1,5
832,1,1,1,1,1,5


In [23]:
# Realizamos la prediccion final como una votacion de las predicciones individuales.
# Si 3 o mas modelos determinan que se fuga la respuesta es fuga.
# Si menos de tres modelos establecen que se fuga determinamos que NO se fuga.
prediccion_final['voto'] = np.where(prediccion_final['Suma'] > 2, 1, 0)

In [24]:
# Analizamos los resultados obtenidos.

matrix_bagging2 = confusion_matrix(y_test, prediccion_final['voto'])
print("\nMatriz de confusion Bagging Aleatorio:")
print(matrix_bagging2)

accuracy_bagging2 = sum(np.diag(matrix_bagging2)) / np.sum(matrix_bagging2)
print(f"Precision Bagging Aleatorio: {accuracy_bagging2:.4f}")

# Lo comparamos con el acierto anterior
print(f"\nPrecision Bagging Manual: {accuracy_bagging1:.4f}")
print(f"Precision Arbol: {accuracy_arbol:.4f}")




Matriz de confusion Bagging Aleatorio:
[[693  16]
 [ 68  57]]
Precision Bagging Aleatorio: 0.8993

Precision Bagging Manual: 0.8849
Precision Arbol: 0.8837


# Realizamos un BOOSTING manual

Los modelos de Boosting son modelos "consecutivos".

Cada modelo aprende de los errores del modelo anterior.

La "forma de aprender" es darle un mayor peso a las observaciones en las que el modelo anterior ha fallado.

Una vez creados los modelos se realizan las predicciones para los datos de train.

Esto es necesario para ajustar los pesos.

El siguiente paso es realizar las predicciones para test.

Tras ello hay que crear un sistema de votacion para determinar la respuesta del modelo.

En este caso entrenamos cada modelo, lo visualizamos y realizamos la prediccion.

In [24]:
from sklearn.tree import DecisionTreeClassifier

pesos = np.ones(len(X_train))
modelo1 = DecisionTreeClassifier(max_depth=2, min_samples_split=10)
modelo1.fit(X_train, y_train, sample_weight=pesos)

y_pred_train1 = modelo1.predict(X_train)
fallos1 = (y_pred_train1 != y_train).astype(int)
pesos += fallos1 * 2

pesos

367     1.0
3103    1.0
549     1.0
2531    1.0
2378    1.0
       ... 
1095    1.0
1130    1.0
1294    1.0
860     1.0
3174    1.0
Name: Churn, Length: 2499, dtype: float64

In [25]:
modelo2 = DecisionTreeClassifier(max_depth=2, min_samples_split=10)
modelo2.fit(X_train, y_train, sample_weight=pesos)

y_pred_train2 = modelo2.predict(X_train)
fallos2 = (y_pred_train2 != y_train).astype(int)
pesos += fallos2 * 2

In [26]:

modelo3 = DecisionTreeClassifier(max_depth=2, min_samples_split=10)
modelo3.fit(X_train, y_train, sample_weight=pesos)

y_pred_train3 = modelo3.predict(X_train)
fallos3 = (y_pred_train3 != y_train).astype(int)
pesos += fallos3 * 2

modelo4 = DecisionTreeClassifier(max_depth=2, min_samples_split=10)
modelo4.fit(X_train, y_train, sample_weight=pesos)

y_pred_train4 = modelo4.predict(X_train)
fallos4 = (y_pred_train4 != y_train).astype(int)
pesos += fallos4 * 2

modelo5 = DecisionTreeClassifier(max_depth=2, min_samples_split=10)
modelo5.fit(X_train, y_train, sample_weight=pesos)

,criterion,'gini'
,splitter,'best'
,max_depth,2
,min_samples_split,10
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [27]:
print(pesos[25:80])

3186    1.0
964     1.0
905     1.0
1517    1.0
3295    1.0
407     1.0
2550    1.0
1511    1.0
2593    1.0
1029    1.0
2236    1.0
2669    1.0
1885    1.0
844     1.0
7       1.0
2176    1.0
1503    1.0
1080    1.0
415     1.0
1178    1.0
1022    1.0
3128    1.0
1735    1.0
3105    1.0
2650    1.0
199     1.0
1200    1.0
1132    1.0
801     1.0
3061    1.0
1356    1.0
283     1.0
1236    1.0
231     1.0
2728    1.0
944     1.0
2715    1.0
96      1.0
2764    1.0
3312    1.0
3158    1.0
3161    1.0
2466    1.0
3040    1.0
3179    1.0
2873    1.0
3196    1.0
1710    1.0
104     1.0
2480    1.0
3214    1.0
912     1.0
2320    1.0
3223    1.0
3071    1.0
Name: Churn, dtype: float64


Podriamos seguir hasta el infinito.

Cuantos mas modelos realicemos mayor riesgo de overfiting.



In [28]:
# Realizamos las predicciones para los datos de test.
prediccion1 = modelo1.predict(X_test)
prediccion2 = modelo2.predict(X_test)
prediccion3 = modelo3.predict(X_test)
prediccion4 = modelo4.predict(X_test)
prediccion5 = modelo5.predict(X_test)

In [30]:
# Una vez realizado esto establecemos un sistema de votacion.
# Esto es algo similar a lo realizado en los modelos anteriores.

In [29]:
prediccion_final = np.column_stack((prediccion1, prediccion2, prediccion3, prediccion4, prediccion5))
prediccion_final = pd.DataFrame(prediccion_final, columns=['Pred1', 'Pred2', 'Pred3', 'Pred4', 'Pred5'])

# Realizamos una suma de las predicciones.
prediccion_final['Suma'] = prediccion_final['Pred1'] + prediccion_final['Pred2'] + prediccion_final['Pred3'] + prediccion_final['Pred4'] + prediccion_final['Pred5']
prediccion_final

,Pred1,Pred2,Pred3,Pred4,Pred5,Suma
0,1,1,1,1,1,5
1,1,1,1,1,1,5
2,1,1,1,1,1,5
3,1,1,1,1,1,5
4,1,1,1,1,1,5
...,...,...,...,...,...,...
829,1,1,1,1,1,5
830,1,1,1,1,1,5
831,1,1,1,1,1,5
832,1,1,1,1,1,5


Realizamos la prediccion final como una votacion de las predicciones individuales.

Si 3 o mas modelos determinan que se fuga la respuesta es fuga.

Si menos de tres modelos establecen que se fuga determinamos que NO se fuga.

In [30]:
prediccion_final['voto'] = np.where(prediccion_final['Suma'] > 2, 1, 0)

Analizamos los resultados.

In [33]:
matrix_boosting = confusion_matrix(y_test, prediccion_final['voto'])
print("\nMatriz de confusion Boosting:")
print(matrix_boosting)

accuracy_boosting = sum(np.diag(matrix_boosting)) / np.sum(matrix_boosting)
print(f"Precision Boosting: {accuracy_boosting:.4f}")


Matriz de confusion Boosting:
[[691  18]
 [ 82  43]]
Precision Boosting: 0.8801


Boosting con el modelo 5

In [31]:
matrix_boosting = confusion_matrix(y_test, modelo5.predict(X_test))
print("\nMatriz de confusion Boosting:")
print(matrix_boosting)

accuracy_boosting2 = sum(np.diag(matrix_boosting)) / np.sum(matrix_boosting)
print(f"Precision Boosting: {accuracy_boosting2:.4f}")


Matriz de confusion Boosting:
[[834]]
Precision Boosting: 1.0000


c:\Users\Dell\PyhtonIA\ML Modelos Ensemble\.venv\Lib\site-packages\sklearn\metrics\_classification.py:534: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Lo comparamos con el acierto anterior

In [34]:
try:
	print(f"Precision Bagging Aleatorio: {accuracy_bagging2:.4f}")
except NameError:
	print("accuracy_bagging2 is not defined.")
try:
	print(f"Precision Bagging Manual: {accuracy_bagging1:.4f}")
except NameError:
	print("accuracy_bagging1 is not defined.")
try:
	print(f"Precision Arbol: {accuracy_arbol:.4f}")
except NameError:
	print("accuracy_arbol is not defined.")
try:
	print(f"Precision Boosting: {accuracy_boosting:.4f}")
except NameError:
	print("accuracy_boosting is not defined.")

accuracy_bagging2 is not defined.
Precision Bagging Manual: 1.0000
Precision Arbol: 1.0000
accuracy_boosting is not defined.
